###  Importing all the required packages

In [1]:
from pprint import pprint
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tensorflow as tf
import keras
from tensorflow.python.framework import ops
#from cnn_utils import *
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import deepdish.io as io
from keras.models import Sequential, Model
#from keras.layers import Input, Add, Dense, Dropout, LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
#from keras.utils import multi_gpu_model
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from itertools import izip
import random
import pandautils as pup
import pandas as pd
from tqdm import tqdm

Using TensorFlow backend.


Welcome to JupyROOT 6.07/07


### Looking inside the dataset

In [2]:
ls /data/debo/jetAnomaly/UdeM_LPS

final_jj_1MEvents.h5


In [3]:
SAMPLE_PATH = os.path.join('/data/debo/jetAnomaly/UdeM_LPS')

In [4]:
os.listdir(SAMPLE_PATH)

['final_jj_1MEvents.h5']

In [5]:
EventInfo = io.load(os.path.join(SAMPLE_PATH,'final_jj_1MEvents.h5'))

/home/debo/env2/local/lib/python2.7/site-packages/tables/group.py:329: UserWarning: leaf ``/objects/e`` is of an unsupported type; it will become an ``UnImplemented`` node
  % self._g_join(childname))
/home/debo/env2/local/lib/python2.7/site-packages/tables/group.py:329: UserWarning: leaf ``/objects/gamma`` is of an unsupported type; it will become an ``UnImplemented`` node
  % self._g_join(childname))
/home/debo/env2/local/lib/python2.7/site-packages/tables/group.py:329: UserWarning: leaf ``/objects/jets/constituents`` is of an unsupported type; it will become an ``UnImplemented`` node
  % self._g_join(childname))
/home/debo/env2/local/lib/python2.7/site-packages/tables/group.py:329: UserWarning: leaf ``/objects/mu`` is of an unsupported type; it will become an ``UnImplemented`` node
  % self._g_join(childname))


In [11]:
EventInfo.keys()

['high_level', 'objects', 'partons']

In [25]:
EventInfo['objects'].keys()

['jets', 'mu', 'e', 'gamma']

### We will only take jets from each event

In [13]:
objects = EventInfo['objects']

In [27]:
objects['jets'].keys()

['constituents', 'obs']

In [44]:
objects['jets']

{'constituents': None,
 'obs': array([[[ 5.39563283e+02, -9.23743352e-01, -1.68751582e+00, ...,
           4.43915892e+00,  3.24084687e+00,  2.17888212e+00],
         [ 4.60895568e+02, -1.33517978e+00,  1.39145153e+00, ...,
           1.15711622e+01,  9.73054123e+00,  7.97088432e+00]],
 
        [[ 7.27789084e+02, -1.65506998e+00,  1.00065047e+00, ...,
           1.41101923e+01,  1.08286705e+01,  9.35908318e+00],
         [ 6.24550581e+02, -1.94968494e-01, -2.15542965e+00, ...,
           2.28335209e+01,  2.01812935e+01,  1.57388792e+01]],
 
        [[ 4.72478786e+02, -1.46958756e+00, -2.33395113e+00, ...,
           9.42582321e+00,  8.68241215e+00,  7.48327303e+00],
         [ 4.48907604e+02,  1.63198306e+00,  6.18280904e-01, ...,
           4.18570518e+01,  3.71974640e+01,  2.80914612e+01]],
 
        ...,
 
        [[ 4.79685628e+02,  1.58962283e+00,  2.86841754e-01, ...,
           9.75549412e+00,  7.92058420e+00,  6.64717293e+00],
         [ 4.06458914e+02,  7.81083118e-01, -1.999

### The 'constituents' are always 'None' and 'obs' seems jets 4momentum

In [31]:
jets = objects['jets']

In [32]:
jets.keys()

['constituents', 'obs']

In [45]:
jets4Mom = jets['obs']

In [46]:
jets4Mom.shape

(705643, 2, 9)

In [48]:
jets4Mom[0,:,:]

array([[539.56328263,  -0.92374335,  -1.68751582,  35.72464581,
         14.73322678,   5.4043684 ,   4.43915892,   3.24084687,
          2.17888212],
       [460.89556826,  -1.33517978,   1.39145153,  42.94837991,
         14.90651512,  12.10266972,  11.57116222,   9.73054123,
          7.97088432]])

### Does it seems each event has 2 jets and 9 variables correspond to pt, eta, phi, m, tau1, tau2, tau3, tau4, tau5 as mentioned in https://gitlab.com/atlas-anomaly-detection/atlas-anomaly-detection-colab/blob/master/data/UdeM-LPS_dataset.md
 So for the dataset I will preparing should I keep only 1st 4 collumns pt, eta, phi, m (converted to E,px,py,pz)and only the leading jet so each entry will then have single jet??